# Gold Reporting Views – Business Analytics Layer

## Purpose
This notebook defines **business-facing reporting views** built on top of the Gold star schema.
These views translate raw facts and dimensions into **decision-ready datasets** by applying
business logic, segmentation rules, and analytical KPIs.

## Why this layer exists
While the Gold layer provides clean **facts and dimensions**, business users rarely consume
raw tables directly. They need:
- Pre-defined KPIs
- Consistent segmentation logic
- Customer and product performance summaries
- Metrics that align with business questions

This layer acts as a **semantic / analytical layer**, ensuring that:
- Business rules are centralized and reusable
- KPI definitions remain consistent across reports
- Power BI dashboards stay lightweight and easy to maintain

## What this layer includes
- Customer-level analytical view (`gold.report_customers`)
- Product-level analytical view (`gold.report_products`)
- Metrics such as:
  - Recency
  - Lifespan
  - Average Order Value / Revenue
  - Monthly Spend / Revenue
  - Customer and product segmentation

## What this layer does NOT do
- No raw data cleansing (handled in Silver)
- No heavy aggregations for storage optimization
- No visualization logic (handled in Power BI)

Power BI connects directly to these views to enable **self-service analytics** while preserving
business logic integrity.


##Create Reporting View: gold.report_customers

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.gold.report_customers AS

WITH base_query AS (
    SELECT
        f.order_number,
        f.product_key,
        f.order_date,
        f.sales_amount,
        f.quantity,
        c.customer_key,
        c.customer_number,
        CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
        FLOOR(months_between(CURRENT_DATE(), c.birthdate) / 12) AS age
    FROM workspace.gold.fact_sales f
    LEFT JOIN workspace.gold.dim_customers c
        ON c.customer_key = f.customer_key
    WHERE f.order_date IS NOT NULL
),

customer_aggregation AS (
    SELECT
        customer_key,
        customer_number,
        customer_name,
        age,
        COUNT(DISTINCT order_number) AS total_orders,
        SUM(sales_amount) AS total_sales,
        SUM(quantity) AS total_quantity,
        COUNT(DISTINCT product_key) AS total_products,
        MAX(order_date) AS last_order_date,
        CAST(months_between(MAX(order_date), MIN(order_date)) AS INT) AS lifespan
    FROM base_query
    GROUP BY
        customer_key,
        customer_number,
        customer_name,
        age
)

SELECT
    customer_key,
    customer_number,
    customer_name,
    age,

    CASE
        WHEN age < 20 THEN 'Under 20'
        WHEN age BETWEEN 20 AND 29 THEN '20–29'
        WHEN age BETWEEN 30 AND 39 THEN '30–39'
        WHEN age BETWEEN 40 AND 49 THEN '40–49'
        ELSE '50 and above'
    END AS age_group,

    CASE
        WHEN lifespan >= 12 AND total_sales > 5000 THEN 'VIP'
        WHEN lifespan >= 12 AND total_sales <= 5000 THEN 'Regular'
        ELSE 'New'
    END AS customer_segment,

    last_order_date,
    CAST(months_between(CURRENT_DATE(), last_order_date) AS INT) AS recency,
    total_orders,
    total_sales,
    total_quantity,
    total_products,
    lifespan,

    CASE
        WHEN total_orders = 0 THEN 0
        ELSE total_sales / total_orders
    END AS avg_order_value,

    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_spend

FROM customer_aggregation;


##Create Reporting View: gold.report_products

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.gold.report_products AS

WITH base_query AS (
    SELECT
        f.order_number,
        f.order_date,
        f.customer_key,
        f.sales_amount,
        f.quantity,
        p.product_key,
        p.product_name,
        p.category,
        p.subcategory,
        p.cost
    FROM workspace.gold.fact_sales f
    LEFT JOIN workspace.gold.dim_products p
        ON f.product_key = p.product_key
    WHERE f.order_date IS NOT NULL
),

product_aggregation AS (
    SELECT
        product_key,
        product_name,
        category,
        subcategory,
        cost,
        CAST(months_between(MAX(order_date), MIN(order_date)) AS INT) AS lifespan,
        MAX(order_date) AS last_sale_date,
        COUNT(DISTINCT order_number) AS total_orders,
        COUNT(DISTINCT customer_key) AS total_customers,
        SUM(sales_amount) AS total_sales,
        SUM(quantity) AS total_quantity,
        ROUND(AVG(sales_amount / NULLIF(quantity, 0)), 1) AS avg_selling_price
    FROM base_query
    GROUP BY
        product_key,
        product_name,
        category,
        subcategory,
        cost
)

SELECT
    product_key,
    product_name,
    category,
    subcategory,
    cost,
    last_sale_date,
    CAST(months_between(CURRENT_DATE(), last_sale_date) AS INT) AS recency_in_months,

    CASE
        WHEN total_sales > 50000 THEN 'High-Performer'
        WHEN total_sales >= 10000 THEN 'Mid-Range'
        ELSE 'Low-Performer'
    END AS product_segment,

    lifespan,
    total_orders,
    total_sales,
    total_quantity,
    total_customers,
    avg_selling_price,

    CASE
        WHEN total_orders = 0 THEN 0
        ELSE total_sales / total_orders
    END AS avg_order_revenue,

    CASE
        WHEN lifespan = 0 THEN total_sales
        ELSE total_sales / lifespan
    END AS avg_monthly_revenue

FROM product_aggregation;
